### Categorização dos produtos 3p em categorias 1p usando Random Forest

Treina-se o modelo com dados 1p usando a descrição como entrada e as categorias martins como saída (label). O modelo treinado é aplicado para, a partir da descrição do produto 3p, predizer a categoria 1p correspondente.

In [0]:
import pandas as pd
import numpy as np
from unicodedata import normalize
from pyspark.sql import functions as F, Window, DataFrame
from pyspark.sql.types import *
from typing import List, Union

import nltk
nltk.download("popular")
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

In [0]:
sdf_seller_product = (
    spark.read
    .option("mergeSchema", True)
    .option("encoding", "ISO-8859-1")
    .csv('/mnt/analyticsquadmkt/data-analytics-projects/3P/INPUT/cadastro_produtos_sellers.csv', sep=";", header=True)
)

sdf_feature_product_comp = (
  spark.read
  .option("mergeSchema", True)
  .parquet("/mnt/advisor-hml/data/01_raw/MIX_ATIVOS_MARTINS/MIX_ATIVOS_MARTINS.parquet")
)

#Produtos mais vendidos em termos de clientes distintos após remoção de similar (período de jan 2022 a jan 2023), com rótulos definidos manualmente
sdf_labels = (
    spark.read
    .option("mergeSchema", True)
    .option("encoding", "UTF-8")
    .csv(f'/mnt/advisor-hml/data/07_model_output/3p_categorization/3p_categorization_labels/labeled_data.csv', sep=";", header=True)
).select('SKU', 'Rank', 'MartinsCategory')

In [0]:
sdf_feature_product_selected = sdf_feature_product_comp.select(F.col('CODPRD').alias('SKU'), F.col('DESCMCMER').alias('ProductDescription'), F.col('DESCTGPRD').alias('ProductCategory'))

#Limpeza dos dados
sdf_product_filtered = sdf_feature_product_selected.filter(F.col("ProductDescription").isNotNull())
sdf_product_filtered = sdf_product_filtered.dropDuplicates(subset=['SKU'])
sdf_product_filtered = sdf_product_filtered.where((F.col('ProductCategory') != 'NÃO INFORMADO')
                                                  & (F.col('ProductCategory') != 'MARTINS DIRETO')
                                                  & (F.col('ProductCategory') != 'KITS SMART')
                                                  & (F.col('ProductCategory') != 'MATERIAL PROMOCIONAL GENERICO')
                                                  & (F.col('ProductCategory') != 'MATERIAL PROMOCIONAL ALIMENTOS/BEBIDAS')
                                                  & (F.col('ProductCategory') != 'MATERIAL PROMOCIONAL BAZAR/PAPELARIA/EQ')
                                                  & (F.col('ProductCategory') != 'MATERIAL PROMOCIONAL BELEZA')
                                                  & (F.col('ProductCategory') != 'MATERIAL PROMOCIONAL ELETRO')
                                                  & (F.col('ProductCategory') != 'MATERIAL PROMOCIONAL GENERICO')
                                                  & (F.col('ProductCategory') != 'MATERIAL PROMOCIONAL HIGIENE')
                                                  & (F.col('ProductCategory') != 'MATERIAL PROMOCIONAL INFORMATICA/TELECOM')
                                                  & (F.col('ProductCategory') != 'MATERIAL PROMOCIONAL MARTCON/AGROVET')
                                                  & (F.col('ProductCategory') != 'SANDALIA') 
                                                 )

'''Observação acerca da Exclusão da categoria "SANDALIA" no modelo Random Forest '''
# RF e LR tendem a escolher uma mesma categoria geralmente com muitos produtos e descrições diversificadas para produtos com descrições diferentes da base de treinamento. Categoria com maior número de SKUs, excluida nesse modelo para evitar que ambos os modelos selecionem a mesma categoria errada na máquina de comitê.

Out[3]: 'Observação acerca da Exclusão da categoria "SANDALIA" no modelo Random Forest '

In [0]:
def remove_accents(sdf: DataFrame, columns: List[str]) -> DataFrame:
    '''Remove acentos das descrições de produtos'''
    
    func_accent = udf(lambda x: normalize('NFKD', x).encode('ASCII', 'ignore').decode('ASCII'), StringType())
    for columns in columns:
        sdf = sdf.withColumn(columns, func_accent(columns))
    
    return sdf  
   
sdf_seller_product = remove_accents(sdf_seller_product, ['DESPRD'])

#Seleciona as colunas de interesse
sdf_seller_product_selected = sdf_seller_product.select(F.col('CODMERSRR').alias('SKU'), F.col('DESPRD').alias('ProductDescription'), F.col('DESCTGPRD').alias('OriginalCategory'))

#Limpeza dos dados
sdf_seller_refined = sdf_seller_product_selected.filter(F.col("ProductDescription").isNotNull())
sdf_seller_refined = sdf_seller_refined.dropDuplicates(subset=['SKU'])

In [0]:
ps = nltk.PorterStemmer()

stopwords_pt = nltk.corpus.stopwords.words('portuguese')
stopwords_en = nltk.corpus.stopwords.words('english')
add_stopwords = ["leve","pague", "(cp)", "[()]", "pq", "desconto", "kg", "pg","lv", "mais","menos","por", "efacil","unidades","unidade","tamanho", "un",  "pequeno", "grande", "extra",  "gde","caixa", "refil", "embalagem", "pedacos", "premium", "new", "mini", "uso", "ate", "multi",  "ultra", "liquido" ,"original", "sabores", "cores", "min", "max", "claro","novo","dark", "brasil", "colorida", "colorido",]
colors =  ["vermelho","vermelha", "laranja", "amarelo","amarela", "verde", "azul", "anil", "indigo", "violeta", "prata",  "preto","branco","branca","preta","cinza","roxo","roxa","rosa","roso","bege","marrom", "red", "black", "blue", "yellow", "white", "green", "purple", "pink", "gold","dourado","dourada"]
stopwords = stopwords_pt + stopwords_en + add_stopwords + colors

df_seller_refined = sdf_seller_refined.toPandas()
df_product_filtered = sdf_product_filtered.toPandas()

def prepare_text(text):
    ''' Remoção de catacteres especiais, números e stopwords, tokenização e stemização'''
    
    text = re.sub(r"[,.;@#?!&$/]+\ *", " ", text) #substituir pontuação por espaço
    text = re.sub("\S*\d\S*", "", text).strip() # remover palavras que contêm números e espaços extras
    text = " ".join([word for word in text.split() if len(word) > 1]) # remover palavras com apenas 1 caractere
    tokens = re.split('\W+', text) # tokenizar
    text = [word for word in tokens if word not in stopwords] #remover stop words e cores
    text1 = []
    for word in text:
      if(word != " " and word != ""  ):
        text1.append(word)
    stemmed=[ps.stem(word) for word in text1] # reduzir palavras por stemização
    return stemmed
  
df_product_filtered['ProductDescriptionToken'] = df_product_filtered['ProductDescription'].apply(lambda x: prepare_text(x.lower()))
df_seller_refined['ProductDescriptionToken'] = df_seller_refined['ProductDescription'].apply(lambda x: prepare_text(x.lower()))

In [0]:
def vectorization(df_product_filtered, df_seller_refined):
  ''' Separação dos dados  e vetorização TFIDF'''
  
  tfidf_vect = TfidfVectorizer(analyzer=prepare_text) #min_df=1
  X_tfidf_train= tfidf_vect.fit_transform(df_product_filtered['ProductDescription'])
  print('train_size:', X_tfidf_train.shape)

  X_tfidf_test =  tfidf_vect.transform(df_seller_refined['ProductDescription'])
  print('test_size:', X_tfidf_test.shape)

  y_train = df_product_filtered['ProductCategory']

  X_tfidf_train_df = pd.DataFrame(X_tfidf_train.toarray())
  X_tfidf_train_df.columns = tfidf_vect.get_feature_names_out()

  X_tfidf_test_df = pd.DataFrame(X_tfidf_test.toarray())
  X_tfidf_test_df.columns = tfidf_vect.get_feature_names_out()
  
  return y_train, X_tfidf_train_df, X_tfidf_test_df

y_train, X_tfidf_train_df, X_tfidf_test_df = vectorization(df_product_filtered, df_seller_refined)

train_size: (22175, 6862)
test_size: (105162, 6862)


In [0]:
rf = RandomForestClassifier(max_features= 'log2', max_depth= 500, n_estimators=200) # create model   
rf.fit(X_tfidf_train_df, y_train) # training model
y_pred = rf.predict(X_tfidf_test_df) #prediction step
probability = rf.predict_proba(X_tfidf_test_df)

In [0]:
prob_values=[]
for prob in probability:
  prob_values.append(np.max(prob))

In [0]:
df_predictions_rf = df_seller_refined.copy()
df_predictions_rf['ProductCategoryRF'] = y_pred
df_predictions_rf['ProbabilityRF'] = prob_values
sdf_predictions_rf = spark.createDataFrame(df_predictions_rf)
df_predictions_rf.drop(columns=['SKU', 'ProductDescriptionToken']).display()

ProductDescription,OriginalCategory,ProductCategoryRF,ProbabilityRF
Raquete Beach Tennis Venum New Contender 3K Full Carbon,acessorios-esportivos,GONDOLA,0.07893020806768301
Copo Termico Venum Red,garrafas-termicas,VIDRO,0.1908424799720635
Copo Termico Venum Green,garrafas-termicas,VIDRO,0.22075166028292742
Copo Termico Venum Black,garrafas-termicas,RECIPIENTES TERMICOS,0.20969540196917077
Copo Termico Venum Yellow,garrafas-termicas,VIDRO,0.2257775828335692
Copo Termico Navalha Red,garrafas-termicas,VIDRO,0.1908424799720635
Isca Soft Borracho Animal Fishing by Johnny Koffmann Junebug,acessorios-p-pesca,DOMISSANITARIO,0.11331793808460836
Isca Soft Cucaracha Animal Fishing by Johnny Hoffmann Junebug,acessorios-p-pesca,DOMISSANITARIO,0.11331793808460836
Isca Soft Cucaracha Animal Fishing by Johnny Hoffmann Manjuba,acessorios-p-pesca,DOMISSANITARIO,0.13183125099195198
Dir-842 D-LINK - Roteador Wireless Gigabit-Ethernet Ac 1200mbps Dual Band 4 Antenas Externas 5dbi,acessorios-e-perifericos,REDE,0.8853782931727148


In [0]:
def create_column_reliable_prediction(sdf: DataFrame, threshold) -> DataFrame: 
  
    sdf = (
        sdf
        .withColumn(
            'ProductCategoryRF', 
            F.when(
                F.col('ProbabilityRF')>= threshold, F.col('ProductCategoryRF')
            )
            .otherwise('NAO ENCONTRADO RF')
        )
    )
    return sdf

sdf_predictions_rf = create_column_reliable_prediction(sdf_predictions_rf, 0.15)

In [0]:
sdf_predictions_rf.where(F.col('ProductCategoryRF')=='NAO ENCONTRADO RF').count()

Out[11]: 35233

In [0]:
#Junção do dataframe de previsões com os dados rotulados
sdf_model_evaluation_rf = sdf_labels.join(sdf_predictions_rf, ['SKU'], how='inner').orderBy(F.col('Rank')).select('SKU','ProductDescription', 'MartinsCategory', 'ProductCategoryRF', 'ProbabilityRF')
#sdf_model_evaluation_rf.display()

#Métricas de desempenho
y = sdf_model_evaluation_rf.select('MartinsCategory').toPandas()
ycalc = sdf_model_evaluation_rf.select('ProductCategoryRF').toPandas()

print('Accuracy:', accuracy_score(y, ycalc))
print('F1_score:', f1_score(y, ycalc, average='weighted'))
print('Precision:', precision_score(y, ycalc, average='weighted'))
print('Recall:', recall_score(y, ycalc, average='weighted'))

Accuracy: 0.8464566929133859
F1_score: 0.8630156157066441
Precision: 0.9078207329346989
Recall: 0.8464566929133859
/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
path = f'/mnt/advisor-hml/data/07_model_output/3p_categorization/RF'

sdf_predictions_rf\
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .format('parquet') \
    .option('header', 'true') \
    .save(path)